## GPU Chooser

This workbook can help rank GPUs according a mixture of features (with the weights determined by the user) and graph it against price.

### Data
Firstly, pull in the parameters from Wikipedia for the cards under consideration (more can easily be added) :

In [14]:
raw="""
name                 | sh:tx:rop | mem | bw|bus|ocl|single|double|watts| amz
GeForce GTX 750 1Gb  | 512:32:16 | 1024| 80|128|1.2| 1044 | 32.6 |  55 | B00IDG3NDY
GeForce GTX 750 2Gb  | 512:32:16 | 2048| 80|128|1.2| 1044 | 32.6 |  55 | B00J3ZNB04
GeForce GTX 750Ti 2Gb| 640:40:16 | 2048| 80|128|1.2| 1306 | 40.8 |  60 | B00IDG3IDO
GeForce GTX 750Ti 4Gb| 640:40:16 | 4096| 80|128|1.2| 1306 | 40.8 |  60 | B00T4RJ8FI
GeForce GTX 760 2Gb  |1152:96:32 | 2048|192|256|1.2| 2257 | 94   | 170 | B00DHW4HXY
GeForce GTX 760 4Gb  |1152:96:32 | 4096|192|256|1.2| 2257 | 94   | 170 | B00E0N49C8

GeForce GTX 960 2Gb  |1024:64:32 | 2048|112|128|1.2| 2308 | 72.1 | 120 | B00SC6HAS4
GeForce GTX 960 4Gb  |1024:64:32 | 4096|112|128|1.2| 2308 | 72.1 | 120 | B00UOYQ5LA
GeForce GTX 970      |1664:104:56| 3584|196|224|1.2| 3494 | 109  | 145 | B00NVODXR4
GeForce GTX 980      |2048:128:64| 4096|224|256|1.2| 4612 | 144  | 165 | B00NT9UT3M
GeForce GTX 980 Ti   |2816:176:96| 6144|336|384|1.2| 5632 | 176  | 250 | B00YNEIAWY
GeForce GTX Titan X  |3072:192:96|12288|336|384|1.2| 6144 | 192  | 250 | B00UXTN5P0

R9 290               |2560:160:64| 4096|320|512|2.0| 4848 | 606  | 275 | B00V4JVY1A
R9 290X              |2816:176:64| 4096|320|512|2.0| 5632 | 704  | 290 | B00FLMKQY2

R9 380 2Gb           |1792:112:32| 2048|182|256|2.1| 3476 | 217  | 190 | B00ZGL8EBK
R9 380 4Gb           |1792:112:32| 4096|182|256|2.1| 3476 | 217  | 190 | B00ZGF3TUC
R9 390               |2560:160:64| 8192|384|512|2.1| 5120 | 640  | 275 | B00ZGL8CYY
R9 390X              |2816:176:64| 8192|384|512|2.1| 5914 | 739  | 275 | B00ZGL8CFI
"""

import re
arr = [ re.split(r'\s*[|:]\s*',l) for l in raw.split('\n')]
headings = arr[1]
data=[ { h:(e if h in 'name.amz' else float(e)) for h,e in zip(headings,a) } for a in arr[2:] ]

Now the GPU card data is in a nice array of dictionary entries, with numeric entries for all but 'name' and the Amazon item ID, indexed in the same order as 'raw'.

### Graph data based on given weights

The concept here is that one can focus on a 'basecard' (for instance, one you already have, or one you've looked at closely), and then assign multiplicative weights to each of a GPU card's qualities, and come up with a 'relative performance' according to that weighting scheme.  Then, performance per dollar can be visualised, against absolute dollar cost (higher is better on the chart).

In [17]:
basecard = 'GeForce GTX 760 2Gb' # Name should match a card with full data above
basedata = [ d for d in data if d['name']==basecard ][0]  

